""""

1. Tried pulling directly from the website source api, got my ass kicked
2. Skywalk API is working and they fixed some requests
3. Testing dashboard combined with Apartments.com data-puller

""""

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html, dash_table
import dash_bootstrap_components as dbc
import pandas as pd



In [2]:
import requests
from bs4 import BeautifulSoup
import json

import gspread
from oauth2client.service_account import ServiceAccountCredentials

f = open("appfolio_access.json")
appfolio_api_key = json.load(f)
payload = {"X-API-KEY" : appfolio_api_key}


## APPFOLIO

In [4]:
from datetime import date

class Appfolio_Report():
    def __init__(self):
        self.today = date.today()
    
    def initialize_spreadsheet(self, spreadsheet_name):
        # use creds to create a client to interact with the Google Drive API
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(creds)

        # Find a workbook by name and open the first sheet
        # Make sure you use the right name here.
        if "docs.google.com" in spreadsheet_name:
            spreadsheet = client.open_by_url(spreadsheet_name)
        else:
            spreadsheet = client.open(spreadsheet_name)
        self.spreadsheet =  spreadsheet

    def connect_worksheet(self):
        """connect to spreadsheet"""
        today=self.today
        #initialize new spreadsheet for each day it is updated
        try:
            sheet = self.spreadsheet.add_worksheet(title=str(today), rows=100, cols=20)
        except gspread.exceptions.APIError:
            sheet = self.spreadsheet.worksheet(str(today))

        #Create Date Updated        
        sheet.batch_update([
            {
                "range" : 'A1:A2', 
                "values" : [["Date Updated"], [str(today)]]
            }
        ])
        self.sheet = sheet
    
    def populate_property_data(self):
        """start pulling information from skywalk appfolio api"""    
        
        sheet = self.sheet
        
        #Pull different properties
        property_request = requests.get('https://api.skywalkapi.com/v1/properties', headers=payload)
        print(property_request.raise_for_status())
        properties = json.loads(property_request.content)
        self.properties = properties
        
        #Loop through apartments
        for apartment in properties["data"]:
            
            #get parameters for specific properties
            self.params = { "search": "?propertyId=" + apartment["appfolioId"],}
            
            #Vacancy Rates
            unit_vacancies_request = requests.get('https://api.skywalkapi.com/v1/unit-vacancies', headers=payload, params = self.params)
            print(unit_vacancies_request.raise_for_status())
            unit_vacancies = json.loads(unit_vacancies_request.content)

            occupancy_rate = round(1 - unit_vacancies["meta"]["count"]/apartment["unitCount"], 3)
            sheet.batch_update([{
                "range" : 'A3:A5', 
                "values" : [[apartment["name"]], ["Occupancy Rate"], [occupancy_rate]]
            }])

            #update unit vacancies     
            unit_vancancy_row_header = ["name", "type", "advertisedRent", "daysVacant", "lastMoveIn", "postedToInternet", "postedToWebsite"]
            sheet.batch_update([{
                "range":"A7:G7", 
                "values" : [unit_vancancy_row_header]
            }])

            unit_row = 8 #STARTS AT 7
            for unit in unit_vacancies["data"]:
                data_to_append = []
                
                for count, col in enumerate(unit_vancancy_row_header):
                    data_to_append.append(unit[col])
                    
                range_rows = "A"+str(unit_row)+":G"+str(unit_row)
                sheet.batch_update([{
                    "range": range_rows,
                    "values": [data_to_append]
                }])

                unit_row = unit_row + 1

report = Appfolio_Report()
report.initialize_spreadsheet("https://docs.google.com/spreadsheets/d/14Loy0p4sV6xMkh1UqFWCqo8296h-4IwzvYi1_sLiGdM/edit#gid=0")
report.connect_worksheet()
report.populate_property_data()

None
None


In [219]:
balance_sheet_request = requests.get('https://api.skywalkapi.com/v1/balance-sheet', headers=payload, params = cf_params)
print(balance_sheet_request.raise_for_status())
balance_sheet = json.loads(balance_sheet_request.content)
balance_sheet

None


{'meta': {'status': 'ok',
  'path': '/v1/balance-sheet',
  'lastUpdated': 1648244034,
  'jobId': 'cd200b3f-587d-4cff-9a05-57b5bf54e3ae',
  'count': 55,
  'query': {'accounting': 'cash', 'date': '03/25/2022'}},
 'links': {},
 'data': [{'id': '0',
   'title': 'ASSETS',
   'glAccountId': None,
   'glAccountName': 'ASSETS',
   'glAccountCode': None,
   'balance': '193,811.91',
   'childIds': ['1',
    '6',
    '7',
    '8',
    '9',
    '10',
    '14',
    '18',
    '19',
    '20',
    '21',
    '22',
    '23',
    '24',
    '25',
    '26',
    '27',
    '28',
    '29',
    '30',
    '31',
    '32']},
  {'id': '1',
   'title': 'Cash',
   'glAccountId': None,
   'glAccountName': 'Cash',
   'glAccountCode': None,
   'balance': '177,226.72',
   'childIds': ['2', '3', '4'],
   'parentId': '0'},
  {'id': '2',
   'title': 'Operating Cash',
   'glAccountId': '2',
   'glAccountName': 'Operating Cash',
   'glAccountCode': '1150',
   'balance': '174,426.72',
   'childIds': [],
   'parentId': '1'},
 

In [224]:
cf_params = { "search": "?propertyId=51&accounting=cash&dateRange=this_month",}

cash_flow_request = requests.get('https://api.skywalkapi.com/v1/cash-flow', headers=payload, params = cf_params)
print(cash_flow_request.raise_for_status())
cashflow = json.loads(cash_flow_request.content)
cashflow

None


{'meta': {'status': 'ok',
  'path': '/v1/cash-flow',
  'lastUpdated': 1648243192,
  'jobId': 'a061d844-a9c3-4de8-92d1-ec62f3bb9fdf',
  'count': 244,
  'query': {'accounting': 'cash',
   'ownerPercentBased': 'No',
   'dateRange': 'this_year_to_date',
   'dateStart': '01/01/2022',
   'dateEnd': '03/25/2022'}},
 'links': {'next': '/v1/cash-flow?accounting=cash&ownerPercentBased=No&dateRange=this_year_to_date&dateStart=01%2F01%2F2022&dateEnd=03%2F25%2F2022&cursor=eyJqb2JJZCI6ImEwNjFkODQ0LWE5YzMtNGRlOC05MmQxLWVjNjJmM2JiOWZkZiIsInBhZ2UiOjJ9'},
 'data': [{'id': '1',
   'title': 'Operating Income',
   'glAccountId': None,
   'glAccountName': 'Income',
   'glAccountCode': None,
   'selectedPeriod': '284,135.48',
   'selectedPeriodAsPercent': '100.00',
   'fiscalYearToDate': '284,135.48',
   'fiscalYearToDateAsPercent': '100.00',
   'childIds': ['2', '3', '8', '9', '10', '11', '12', '51', '52']},
  {'id': '2',
   'title': 'Appfolio Mtm Additional Fee',
   'glAccountId': '112',
   'glAccountName'

# Dashboard

In [3]:
def restack_frame(df, target_variable):
    # restacking frame for merging
    
    df.drop(columns= ["RegionID", "SizeRank", "RegionType", "StateName"], inplace=True)

    reframed_rent_df = pd.DataFrame(df.set_index("RegionName").stack()).reset_index()
    reframed_rent_df.columns = ["RegionName", "Date", target_variable]
    reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")
    reframed_rent_df.Date = reframed_rent_df.Date.str[:7]
    
    return reframed_rent_df

In [51]:
# ZRent from Zillow
rent_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_zori_sm_sa_month.csv?t=1663774133")
reframed_rent_df = restack_frame(rent_df, "ZRent")

# housing from Zillow
housing_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1663287638")
reframed_housing_df = restack_frame(housing_df, "ZEstimate")

# inventory from Zillow
inventory_df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/invt_fs/Metro_invt_fs_uc_sfrcondo_sm_month.csv?t=1663287638")
reframed_inventory_df = restack_frame(inventory_df, "Inventory")

# combining rent and housing
reframe_df = pd.merge(reframed_rent_df, reframed_housing_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])
reframe_df = pd.merge(reframe_df, reframed_inventory_df, how = "left", left_on = ["RegionName", "Date"], right_on = ["RegionName", "Date"])

reindexed_reframed_df = reframe_df.set_index(["RegionName", "Date"])


# rent buy ratio
reindexed_reframed_df["rent_buy_ratio"] = reindexed_reframed_df["ZRent"]/reindexed_reframed_df["ZEstimate"]

# adding percentage change
reindexed_reframed_df["rent_pct_change"] = reindexed_reframed_df["ZRent"].astype(float).pct_change()
reindexed_reframed_df["home_value_pct_change"] = reindexed_reframed_df["ZEstimate"].pct_change()
reindexed_reframed_df["inventory_pct_change"] = reindexed_reframed_df["Inventory"].pct_change()

reindexed_reframed_df["rent_buy_pct_change"] = reindexed_reframed_df["rent_buy_ratio"].pct_change()

# removing wrong months
reindexed_reframed_df.reset_index(inplace = True)
reindexed_reframed_df = reindexed_reframed_df[reindexed_reframed_df.Date != "2015-03"]

reindexed_reframed_df.head()

,RegionName,Date,ZRent,ZEstimate,Inventory,rent_buy_ratio,rent_pct_change,home_value_pct_change,inventory_pct_change,rent_buy_pct_change
1,United States,2015-04,1402.345978,195029.0,NaN,0.007190,0.005757,0.004864,NaN,0.000889
2,United States,2015-05,1409.359291,195966.0,NaN,0.007192,0.005001,0.004804,NaN,0.000196
3,United States,2015-06,1416.071874,196775.0,NaN,0.007196,0.004763,0.004128,NaN,0.000632
4,United States,2015-07,1420.171658,197486.0,NaN,0.007191,0.002895,0.003613,NaN,-0.000715
5,United States,2015-08,1425.425977,198167.0,NaN,0.007193,0.003700,0.003448,NaN,0.000251


## Census Data

In [26]:
import sys
sys.path.extend(["/Users/nelsonlin/Documents/GitHub/RealEstate/src"])
import interpolation as itrp

import censusdata
import us
import pandas as pd
import importlib

importlib.reload(censusdata)

censusdata.search('acs5', 2019, 'concept', 'total population')
#censusdata.printtable(censusdata.censustable('acs5', 2019, 'B23025'))

population_in_occupied_units = "B01003_001E"
median_value = "B25077_001E"
gross_rent_total = "B25031_001E"
median_income = "B19013_001E"

census_columns_to_download = [population_in_occupied_units, median_value, gross_rent_total, median_income]
years_to_download = [2015, 2016, 2017, 2018, 2019]

aggregate_census_df = pd.DataFrame()

for year in years_to_download:
    #or state in us.states.STATES:
    new_data_df = censusdata.download('acs5', year, censusdata.censusgeo([("metropolitan statistical area/micropolitan statistical area", "*")]), census_columns_to_download)
    us_data_df = censusdata.download('acs5', year, censusdata.censusgeo([("us", "*")]), census_columns_to_download)
    new_data_df = pd.concat([new_data_df, us_data_df])

    new_data_df["year"] = year
    aggregate_census_df = aggregate_census_df.append(new_data_df, ignore_index=False)

# cleaning up
new_aggregate_census_df = aggregate_census_df.reset_index()
new_aggregate_census_df.columns = ["RegionName", "population", "median_value", "gross_rent_total", "median_income", "year"]
new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.astype(str).apply(lambda x: x[:-98].replace(",", ""))

# recoding names
new_aggregate_census_df.loc[new_aggregate_census_df[new_aggregate_census_df["RegionName"] == ""].index, "RegionName"] = "United States"

to_replace_cities = {
    "Austin-Round Rock TX": "Austin TX",
    "Austin-Round Rock-Georgetown TX": "Austin TX",
    "Cleveland-Elyria OH": "Cleveland OH",
    "Grand Rapids-Wyoming MI": "Grand Rapids MI",
    "Grand Rapids-Kentwood MI" : "Grand Rapids MI",
    "Miami-Fort Lauderdale-Pompano Beach FL": "Miami-Fort Lauderdale FL",
    "Miami-Fort Lauderdale-West Palm Beach FL": "Miami-Fort Lauderdale FL",
    "New Orleans-Metairie LA": "New Orleans LA",
    "Raleigh-Cary NC": "Raleigh NC",
    "San Diego-Carlsbad CA": "San Diego CA",
    "Stockton-Lodi CA": "Stockton CA",
    "Milwaukee-Waukesha WI": "Milwaukee WI",
    "Birmingham-Hoover AL" :  "Birmingham AL",
    "New Haven-Milford CT" : "New Haven CT",
    "Greenville-Anderson SC" : "Greenville SC",
    "Provo-Orem UT" : "Provo UT",
    "San Francisco-Oakland-Hayward CA" : "San Francisco CA",
    "San Francisco-Oakland-Berkeley CA" : "San Francisco CA",
    "San Antonio-New Braunfels TX" : "San Antonio TX",
    "Indianapolis-Carmel-Anderson IN" : "Indianapolis IN",
    "Buffalo-Cheektowaga NY" : "Buffalo NY",
    "Dayton-Kettering OH" : "Dayton OH",
    "Lakeland-Winter Haven FL" : "Lakeland FL",
    "Des Moines-West Des Moines IA" : "Des Moines IA",
    "Dallas-Fort Worth-Arlington TX" : "Dallas-Fort Worth TX",
    "Cape Coral-Fort Myers FL": "Fort Myers FL",
    "Greensboro-High Point NC" : "Greensboro NC",
    "Detroit-Warren-Dearborn MI": "Detroit MI",
    "Riverside-San Bernardino-Ontario CA": "Riverside CA",
    "Phoenix-Mesa-Scottsdale AZ": "Phoenix AZ",
    "Phoenix-Mesa-Chandler AZ": "Phoenix AZ",
    "Seattle-Tacoma-Bellevue WA": "Seattle WA",
    "San Diego-Chula Vista-Carlsbad CA": "San Diego CA",
    "Baltimore-Columbia-Towson MD" : "Baltimore MD",
    "Denver-Aurora-Lakewood CO" : "Denver CO",
    "Sacramento--Roseville--Arden-Arcade CA" : "Sacramento CA",
    "Sacramento-Roseville-Folsom CA" : "Sacramento CA",
    "Orlando-Kissimmee-Sanford FL" : "Orlando FL",
    "Las Vegas-Henderson-Paradise NV" : "Las Vegas NV",
    "Santa Rosa-Petaluma CA" : "Santa Rosa CA",
    "Bridgeport-Stamford-Norwalk CT" : "Stamford CT",
    "Albany-Schenectady-Troy NY" : "Albany NY",
    "Greenville-Anderson-Mauldin SC" : "Greenville SC",
    "McAllen-Edinburg-Mission TX" : "McAllen TX",
    "Little Rock-North Little Rock-Conway AR" : "Little Rock AR",
    "Charleston-North Charleston SC" : "Charleston SC",
    "Ogden-Clearfield UT" : "Ogden UT",
    "Deltona-Daytona Beach-Ormond Beach FL" : "Daytona Beach FL",
    "Durham-Chapel Hill NC" : "Durham NC",
    "Houston-The Woodlands-Sugar Land TX" : "Houston TX",
    "Atlanta-Sandy Springs-Roswell GA" : "Atlanta GA",
    "Atlanta-Sandy Springs-Alpharetta GA" : "Atlanta GA",
    "Hartford-West Hartford-East Hartford CT" : "Hartford CT",
    "Hartford-East Hartford-Middletown CT" : "Hartford CT",
    "Nashville-Davidson--Murfreesboro--Franklin TN" : "Nashville TN",
    "Houston-The Woodlands-Sugar Land TX" : "Houston TX",
    "Tampa-St. Petersburg-Clearwater FL" : "Tampa FL",
    "San Jose-Sunnyvale-Santa Clara CA" : "San Jose CA",
    "Milwaukee-Waukesha-West Allis WI" : "Milwaukee WI",
    "Buffalo-Cheektowaga-Niagara Falls NY" : "Buffalo NY",
    "Oxnard-Thousand Oaks-Ventura CA" : "Ventura CA",
    "New York-Newark-Jersey City NY-NJ-PA" : "New York NY",
    "Chicago-Naperville-Elgin IL-IN-WI" : "Chicago IL",
    "Philadelphia-Camden-Wilmington PA-NJ-DE-MD" : "Philadelphia PA",  
    "Washington-Arlington-Alexandria DC-VA-MD-WV" : "Washington DC",
    "Boston-Cambridge-Newton MA-NH" : "Boston MA",
    "St. Louis MO-IL" : "St. Louis MO",
    "Portland-Vancouver-Hillsboro OR-WA" : "Portland OR",
    "Charlotte-Concord-Gastonia NC-SC" : "Charlotte NC",
    "Cincinnati OH-KY-IN" : "Cincinnati OH",
    "Virginia Beach-Norfolk-Newport News VA-NC" : "Virginia Beach VA",
    "Providence-Warwick RI-MA" : "Providence RI",
    "Memphis TN-MS-AR" : "Memphis TN",
    "Omaha-Council Bluffs NE-IA" : "Omaha NE",
    "Allentown-Bethlehem-Easton PA-NJ" : "Allentown PA",
    "Worcester MA-CT" : "Worcester MA",
    "Kansas City MO-KS" : "Kansas City MO",
    "Louisville/Jefferson County KY-IN" : "Louisville-Jefferson County KY",
    "Minneapolis-St. Paul-Bloomington MN-WI" : "Minneapolis-St Paul MN",
}

for city in to_replace_cities:
    new_aggregate_census_df.RegionName = new_aggregate_census_df.RegionName.str.replace(city, to_replace_cities[city])

    
def interpolate_dataframe(new_aggregate_census_df, final_data_year, interpolation_year):
    """
    Requires RegionName
    Interpolates into 2022
    """
    
    interpolated_census = []

    for region in new_aggregate_census_df["RegionName"].unique():

        # select specific region
        test_pd = new_aggregate_census_df[new_aggregate_census_df["RegionName"] == region]

        # add nan so it predicts into future years
        added_rows = pd.DataFrame({"year": range(final_data_year,interpolation_year  + 1)})
        combined_df = pd.concat([test_pd, added_rows]).reset_index(drop=True)
        combined_df = combined_df.set_index("year")

        # Interpolate for each column except the text one
        for col in combined_df.columns:
            if col != "RegionName":
                combined_df[col] = itrp.poly_degree2_trend(combined_df.loc[:, col])["adoption"]

        # Pad nan region name
        combined_df["RegionName"].interpolate(method = "pad", inplace=True)
        interpolated_census.append(combined_df)

    return pd.concat(interpolated_census)

full_interpolated_census_data = interpolate_dataframe(new_aggregate_census_df, final_data_year = 2020, interpolation_year = 2022).to_csv("data/fully_interpolated_census_data_msa.csv")


/var/folders/2x/1y29_dkx10zcpzr8lh9g1w_h0000gn/T/ipykernel_70846/153359896.py:32: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2x/1y29_dkx10zcpzr8lh9g1w_h0000gn/T/ipykernel_70846/153359896.py:32: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2x/1y29_dkx10zcpzr8lh9g1w_h0000gn/T/ipykernel_70846/153359896.py:32: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2x/1y29_dkx10zcpzr8lh9g1w_h0000gn/T/ipykernel_70846/153359896.py:32: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/2x/1y29_dkx10zcpzr8lh9g1w_h0000gn/T/ipykernel_70846/153359896.py:32: FutureWarning:

The frame.append method is depreca

AttributeError: module 'interpolation' has no attribute 'poly_degree2_trend'

In [5]:
census_data = pd.read_csv("https://raw.githubusercontent.com/nelsonlin2708968/RealEstate/master/data/fully_interpolated_census_data_msa.csv")
census_data.rename(columns = {"year" : "Date"}, inplace=True)
census_data["population_change"] = census_data["population"].pct_change()
census_data["median_income_change"] = census_data["median_income"].pct_change()

In [6]:
len(census_data[census_data["population"] > 100000]["RegionName"].unique())

437

# Plotting

In [52]:
def graph_layout(id_name, title):
    return html.Div(children=[
        
                html.Br(),
                html.Label(title),
                dcc.Graph(
                    id=id_name,
                )],
            style=style, draggable=True)

def create_sub_plot(filtered_df, dependent_variable):
    fig = px.line(filtered_df, x="Date", y=dependent_variable, color="RegionName", hover_name="RegionName")
    fig.update_layout(transition_duration=500, template = template)
    return fig

In [53]:
app = JupyterDash(__name__, 
            external_stylesheets=[dbc.themes.BOOTSTRAP],
            meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"}
    ],)
del app.config._read_only["requests_pathname_prefix"]
server = app.server


template = "simple_white"
style = {'width': '49%', 'display': 'inline-block',"text-align" : "center",}
 

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reindexed_reframed_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    graph_layout('Zrent', 'Graph of ZRents'), 
    graph_layout('Zrent_pct_change', 'Graph of ZRents Change'), 
    
    graph_layout('ZEst', 'Graph of ZEstimates'), 
    graph_layout('ZEst_pct_change', 'Graph of ZEstimates Change'),
    
    graph_layout('Zrent_buy', 'Graph of ZRent/ZBuy'), 
    graph_layout('Zrent_buy_pct_change', 'Graph of ZRent/ZBuy Change'),
    
    graph_layout('Inventory', 'Graph of Inventory'), 
    graph_layout('inventory_pct_change', 'Inventory Rate of Change'),
    
    graph_layout(id_name = 'median_income', title = 'Incomes (Projected 2020-2022)'), 
    graph_layout(id_name = 'median_income_change', title = 'Income Growth (Projected 2020-2022)'), 

    graph_layout(id_name = 'population', title = 'Population (Projected 2020-2022)'),
    graph_layout(id_name = 'population_change', title = 'Population Growth (Projected 2020-2022)'),
    
    
    ])

@app.callback(
    Output('Zrent', 'figure'),
    Output('Zrent_pct_change', 'figure'),
    Output('ZEst', 'figure'),
    Output('ZEst_pct_change', 'figure'),
    Output('Zrent_buy', 'figure'),
    Output('Zrent_buy_pct_change', 'figure'),
    Output('Inventory', 'figure'),
    Output('inventory_pct_change', 'figure'),
    Output('median_income', 'figure'),
    Output('median_income_change', 'figure'),
    Output('population', 'figure'),
    Output('population_change', 'figure'),

    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reindexed_reframed_df[reindexed_reframed_df.RegionName.isin(selected_cities)]  
    census_filtered_df = census_data[census_data.RegionName.isin(selected_cities)]  

    fig1 = create_sub_plot(filtered_df, "ZRent")
    fig2 = create_sub_plot(filtered_df, "rent_pct_change")
    
    fig3 = create_sub_plot(filtered_df, "ZEstimate")
    fig4 = create_sub_plot(filtered_df, "home_value_pct_change")
    
    fig5 = create_sub_plot(filtered_df, "rent_buy_ratio")
    fig6 = create_sub_plot(filtered_df, "rent_buy_pct_change")
    
    fig7 = create_sub_plot(filtered_df[(filtered_df["RegionName"] != "United States") & (filtered_df["Date"] > "2018-01")], "Inventory")
    fig8 = create_sub_plot(filtered_df[filtered_df["Date"] > "2018-01"], "inventory_pct_change")
    
    fig9 = create_sub_plot(census_filtered_df, "median_income")
    fig10 = create_sub_plot(census_filtered_df[census_filtered_df["Date"] != 2015], "median_income_change")
    
    fig11 = create_sub_plot(census_filtered_df[census_filtered_df["RegionName"] != "United States"], "population")
    fig12 = create_sub_plot(census_filtered_df[census_filtered_df["Date"] != 2015], "population_change")
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9, fig10, fig11, fig12
    
if __name__ == '__main__':
    app.run_server(debug=False, port = 8060)

 * Running on http://127.0.0.1:8060
Press CTRL+C to quit
127.0.0.1 - - [21/Sep/2022 10:30:33] "GET /_alive_d0535a24-8686-4c9a-bfe2-510e58736b6f HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8060/


127.0.0.1 - - [21/Sep/2022 10:30:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_1m1663287394.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/deps/react@16.v2_6_1m1663287394.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_1m1663287394.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_1m1663287394.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1663287568.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_1m1663287394.min.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Sep/2022 10:30:36] "GET /_dash-component-suites/dash/dcc/dash_core_comp

selected_cities ['United States', 'Jacksonville FL']


127.0.0.1 - - [21/Sep/2022 10:30:37] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [21/Sep/2022 10:34:10] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'Miami FL', 'Phoenix AZ', 'Orlando FL', 'Tampa FL']


127.0.0.1 - - [21/Sep/2022 10:34:33] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'Miami FL', 'Phoenix AZ', 'Tampa FL']


127.0.0.1 - - [21/Sep/2022 10:41:49] "POST /_dash-update-component HTTP/1.1" 200 -


,RegionName,Date,ZRent,ZEstimate,Inventory,rent_buy_ratio,rent_pct_change,home_value_pct_change,inventory_pct_change,rent_buy_pct_change
125,New York NY,2018-02,2676.216666,458848.0,69165.0,0.005832,0.002054,0.005718,-0.002998,-0.003644
126,New York NY,2018-03,2682.242060,461794.0,73707.0,0.005808,0.002251,0.006420,0.065669,-0.004142
127,New York NY,2018-04,2697.452273,464820.0,80345.0,0.005803,0.005671,0.006553,0.090059,-0.000876
128,New York NY,2018-05,2714.530415,467195.0,85864.0,0.005810,0.006331,0.005110,0.068691,0.001215
129,New York NY,2018-06,2730.709611,468849.0,90067.0,0.005824,0.005960,0.003540,0.048950,0.002411
...,...,...,...,...,...,...,...,...,...,...
47835,Portales NM,2022-04,873.774770,127946.0,18.0,0.006829,0.007755,0.006529,0.000000,0.001218
47836,Portales NM,2022-05,871.571484,128438.0,20.0,0.006786,-0.002522,0.003845,0.111111,-0.006343
47837,Portales NM,2022-06,896.464771,129572.0,23.0,0.006919,0.028561,0.008829,0.150000,0.019560
47838,Portales NM,2022-07,932.777778,130876.0,25.0,0.007127,0.040507,0.010064,0.086957,0.030140


In [ ]:
from flask import request
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    

# BLS

In [214]:
import requests
import json
import prettytable
headers = {'Content-type': 'application/json'}
series_id = 'CUUR0000SA0'
data = json.dumps({"seriesid": [series_id],"startyear":"2011", "endyear":"2021"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
series = json_data['Results']['series'][0]

pd.DataFrame(series["data"])

,year,period,periodName,value,footnotes
0,2020,M12,December,260.474,[{}]
1,2020,M11,November,260.229,[{}]
2,2020,M10,October,260.388,[{}]
3,2020,M09,September,260.280,[{}]
4,2020,M08,August,259.918,[{}]
...,...,...,...,...,...
115,2011,M05,May,225.964,[{}]
116,2011,M04,April,224.906,[{}]
117,2011,M03,March,223.467,[{}]
118,2011,M02,February,221.309,[{}]


### Add apartment list too

In [122]:
apt_list_df = pd.read_csv("https://assets.ctfassets.net/jeox55pd4d8n/2QcueWduZDvVORgKJtXveW/407262141f9b1669748e6e147f112c13/Apartment_List_Rent_Estimates_City_2022_2.csv")
apt_list_df.dropna(inplace=True)

In [124]:
apt_list_df.drop("FIPS_Code", "Population")

,City_Name,FIPS_Code,Population,Bedroom_Size,2017_01,2017_02,2017_03,2017_04,2017_05,2017_06,...,2021_05,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12,2022_01,2022_02
0,"Birmingham, AL",107000.0,212021.0,_Overall,825.00,829.00,835.00,843.00,852.00,854.00,...,961.00,980.00,1007.00,1019.00,1026.00,1019.00,1014.00,1019.00,1031.00,1041.00
1,"Birmingham, AL",107000.0,212021.0,_Studio,730.00,734.00,739.00,746.00,753.00,756.00,...,850.00,867.00,890.00,902.00,907.00,902.00,897.00,901.00,912.00,921.00
2,"Birmingham, AL",107000.0,212021.0,1br,693.00,697.00,702.00,709.00,716.00,718.00,...,808.00,824.00,846.00,857.00,862.00,857.00,852.00,856.00,867.00,875.00
3,"Birmingham, AL",107000.0,212021.0,2br,816.00,821.00,827.00,834.00,843.00,846.00,...,952.00,970.00,996.00,1009.00,1015.00,1009.00,1004.00,1008.00,1021.00,1031.00
4,"Birmingham, AL",107000.0,212021.0,3br,1066.00,1072.00,1079.00,1089.00,1101.00,1104.00,...,1243.00,1267.00,1301.00,1317.00,1325.00,1317.00,1310.00,1317.00,1332.00,1346.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,"Wauwatosa, WI",5584675.0,47882.0,_Studio,NA,NA,NA,980.00,1004.00,992.00,...,960.00,999.00,1046.00,1058.00,1056.00,1020.00,997.00,1003.00,1015.00,1028.00
3818,"Wauwatosa, WI",5584675.0,47882.0,1br,NA,NA,NA,1140.00,1167.00,1153.00,...,1116.00,1162.00,1216.00,1230.00,1228.00,1186.00,1159.00,1166.00,1180.00,1195.00
3819,"Wauwatosa, WI",5584675.0,47882.0,2br,NA,NA,NA,1222.00,1250.00,1235.00,...,1196.00,1245.00,1303.00,1318.00,1316.00,1271.00,1242.00,1250.00,1265.00,1281.00
3820,"Wauwatosa, WI",5584675.0,47882.0,3br,NA,NA,NA,1495.00,1530.00,1512.00,...,1464.00,1524.00,1595.00,1613.00,1610.00,1556.00,1520.00,1530.00,1548.00,1568.00


In [ ]:
reframed_rent_df = pd.DataFrame(rent_df.set_index("RegionName").stack()).reset_index()
reframed_rent_df.columns = ["RegionName", "Date", "ZRent"]
reframed_rent_df.RegionName = reframed_rent_df.RegionName.str.replace(",", "")

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

import ApartmentsScraperHelper

bb = "xtqrw9_u9Isz8svW"
# pull the same code everytime but for San Marco
ApartmentsScraperHelper.main(bb = bb)

pd.read_csv(bb+".csv")

NameError: name 'csv' is not defined